In [1]:
data = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}""".splitlines()

In [ ]:
from pathlib import Path

data = Path("input").read_text().strip().splitlines()

In [2]:
def get_required_lights(s: str) -> list[bool]:
    return [v == "#" for v in s.removeprefix("[").removesuffix("]")]

def get_buttons(d: list[str]) -> list[tuple[int]]:
    return [
        tuple(
            [
                int(n)
                for n in part.removeprefix("(").removesuffix(")").split(",")
            ]
        )
        for part in d
    ]

def get_required_joltage(s: str) -> list[int]:
    return [int(n) for n in s.removeprefix("{").removesuffix("}").split(",")]

In [3]:
class LightsMachine:
    def __init__(self, required_lights: list[bool], buttons: list[tuple[int]]):
        self.required_lights = required_lights
        self.buttons = buttons
        self.reset()

    @classmethod
    def from_data(cls, row: str):
        a, *b, c = row.split(" ")
        required_lights = get_required_lights(a)
        buttons = get_buttons(b)
        return cls(required_lights, buttons)
    
    def press_button(self, button: tuple[int]):
        for i in button:
            self.lights[i] = not self.lights[i]

    def is_configured(self):
        return self.lights == self.required_lights

    def reset(self):
        self.lights = [False for light in self.required_lights]

In [4]:
lights_machines = [LightsMachine.from_data(row) for row in data]

In [5]:
from itertools import combinations

def get_shortest_lights_combo(machine: LightsMachine):
    r = 0
    while True:
        r += 1
        for combo in combinations(machine.buttons, r=r):
            for btn in combo:
                machine.press_button(btn)
            if machine.is_configured():
                machine.reset()
                return r
            machine.reset()

In [6]:
print("Part 1:")
print(sum(get_shortest_lights_combo(machine) for machine in lights_machines))

Part 1:
7


In [40]:
class JoltageOverflow(Exception):
    pass

class JoltageMachine:
    def __init__(self, buttons: list[tuple[int]], required_joltage: list[int]):
        self.required_joltage = required_joltage
        self.buttons = buttons
        self.reset()

    @classmethod
    def from_data(cls, row: str):
        a, *b, c = row.split(" ")
        buttons = get_buttons(b)
        joltage = get_required_joltage(c)
        return cls(buttons, joltage)
    
    def press_button(self, button: tuple[int]):
        for i in button:
            self.joltage[i] -= 1
        if any(j < 0 for j in self.joltage):
            raise JoltageOverflow

    def is_configured(self):
        return all(j == 0 for j in self.joltage)

    def reset(self):
        self.joltage = list(self.required_joltage)

In [41]:
joltage_machines = [JoltageMachine.from_data(row) for row in data]

In [ ]:
from itertools import combinations_with_replacement

def get_joltage_combos(machine: JoltageMachine, joltage_index: int):
    required_joltage = machine.joltage[joltage_index]
    buttons = [btn for btn in machine.buttons if joltage_index in btn]
    return combinations_with_replacement(buttons, r=required_joltage)

def get_smallest_target_joltage(machine: JoltageMachine):
    j = min([j for j in machine.joltage if j > 0])
    return machine.joltage.index(j)

def get_shortest_joltage_combo(machine: JoltageMachine, presses: int = 0):
    # if machine.is_configured():
    #     return presses
    ji = get_smallest_target_joltage(machine)
    for presses, combo in enumerate(get_joltage_combos(machine, ji), start=1):
        for btn in combo:
            try:
                machine.press_button(btn)
                print(machine.joltage)
            except JoltageOverflow:
                machine.reset()
                continue
        
        print(">>>", machine.joltage)
    return min(get_shortest_joltage_combo(machine, presses))


m = joltage_machines[0]
m.reset()
min(get_shortest_joltage_combo(m))

[2, 5, 3, 7]
[1, 5, 2, 7]
[0, 5, 1, 7]
>>> [0, 5, 1, 7]
[2, 5, 3, 7]
[1, 4, 3, 7]
>>> [1, 4, 3, 7]
[0, 4, 2, 7]
[2, 4, 4, 7]
>>> [2, 4, 4, 7]
[1, 3, 4, 7]
[0, 2, 4, 7]
>>> [3, 5, 4, 7]


TypeError: 'NoneType' object is not iterable

In [ ]:
print("Part 2:")
print(sum(get_shortest_joltage_combo(machine) for machine in joltage_machines))